# 応用演習

基礎演習が終わった方から、応用問題に取り組んでみてください。

解答は本Notebookの最後に付属していますが、できるだけ解答を見ずにチャレンジしてください。

## Playbookの再利用

### 問題

ここまでの基礎演習の内容を利用して、以下の作業を [advanced/01_site.yml](/edit/aitac-automation-handson/advanced/01_site.yml) として1つのPlaybookから実行できるようにしてください。

* 2台のインスタンスを起動する（基礎演習１）
* 起動したインスタンスをインベントリへ追加する（基礎演習6）→ [advanced/01_add_host_and_wait.yml](/edit/aitac-automation-handson/advanced/01_add_host_and_wait.yml)
* 2台のインスタンスがログイン可能になるのを待つ（[wait_for](http://docs.ansible.com/ansible/latest/wait_for_module.html)を使う   → [advanced/01_add_host_and_wait.yml](/edit/aitac-automation-handson/advanced/01_add_host_and_wait.yml)）
* 起動したインスタンスへhttpdをインストールし、index.htmlを配置してから起動する（基礎演習３）
* 設定した内容が正しく動作しているかテストする（基礎演習４）

In [ ]:
# 実行例

ansible-playbook advanced/01_site.yml

### ヒント

* import_playbook でここまでに利用した Playbook をそのまま利用する。
* 一部のPlaybookは環境変数としていくつかの値を与えていたので、別の方法でPlaybookに変数を与える必要がある。
* EC2インスタンスの起動時に、結果をec2をいう変数に格納しているので、これを使って add_host を行い、インベントリを生成する。
* 移動したばかりのインスンタンスはすぐにはログインできないので、 wait_for を使って SSH ログイン可能になるのを待つ必要がある（60秒程度）

### 注意

* 次の問いに移る前に、[演習サーバーの削除](/notebooks/aitac-automation-handson/05_delete_target_servers.ipynb)を参考に、起動した演習サーバーを削除しておいてください。
* 演習に利用しているスペックの低いインスタンスでは、インスタンスの起動 → yum モジュールの操作を連続して行うと、yum の部分で非常に時間がかかることがあります。
* 一時的に性能を上げて試したい場合は、Playbookで指定している インスタンスタイプを `m4.large` に、ボリュームタイプを `gp2`、ボリュームサイズを `300GB` すると性能が向上します。

## role 化

### 問題

応用演習１で使用した各Playbookを role 化し、より汎用性を高めてください。作成する role は以下になります。

* advanced/roles/ec2_instance_create
* advanced/roles/wait_server_boot
* advanced/roles/httpd_on
* advanced/roles/httpd_on_test

これらの作成した role を [advanced/02_site.yml](/edit/aitac-automation-handson/advanced/02_site.yml)  として実行できるようにしてください。

In [ ]:
# 実行例

ansible-playbook advanced/02_site.yml

### ヒント

* 先の演習で add_host と ec2_instance_create ロールに含めると処理がわかりやすくなります。

### 注意

* ここでの環境は引き続き利用するので、この段階では削除しないようにお願いします。
* 先の演習と同様に、性能を上げたい場合はインスタンスタイプ、ボリュームサイズに大きめの値を設定してください。

## ロードバランサーの設定

### 問題

応用演習2で構築した環境に対してHTTPの負荷分散を行うように設定してください。負荷分散には Nginxを利用し、応用演習2で構築したサーバー上に同居するように設定します。この時にNginxが待ち受けるポートは8080となるように設定します。

動作要件

* nginx をインストールする
* nginx は起動されるとポート8080で待ち受ける
* 受け付けたアクセスは、応用演習2で起動した2台のHTTPDへのロードバランス処理を行う
* バランシングが行われている事をテストする。

以下の role を作成してください。

* advanced/roles/nginx_lb
* advanced/roles/nginx_lb_test

作成した role を [advanced/03_site.yml](/edit/aitac-automation-handson/advanced/03_site.yml)  として実行できるようにしてください。この時にはダイナミックインベントリを利用して対象を指定してください。

In [ ]:
# 実行例

ansible-playbook -i inventories/ec2.py --private-key aitac-automation-key.pem -u centos solutions/03_site.yml

### ヒント

* nginx のインストールには EPEL を有効にする必要がある（epel-release のインストール → nginx のインストール と順番が重要です）
* nginx の設定ファイルは template モジュールを利用する
* バランシングの確認には shellモジュールや url モジュールと assert の組み合わせて行う。

### 注意

* 次の問いに移る前に、[演習サーバーの削除](/notebooks/aitac-automation-handson/05_delete_target_servers.ipynb)を参考に、起動した演習サーバーを削除しておいてください。

## 全ての設定作業を連結

### 問題

応用演習2,3 で実施した内容を全て連結して [advanced/04_site.yml](/edit/aitac-automation-handson/advanced/04_site.yml) として実行できるようにしてください。

* インスタンスの起動
* HTTPD のインストールと設定、テスト
* Nginx のインストールと設定、テスト

In [ ]:
# 実行例

ansible-playbook advanced/04_site.yml

応用演習は以上となります。

## 解答例

### 解答例

* [solutions/01_site.yml](/edit/aitac-automation-handson/solutions/01_site.yml)
* [solutions/01_add_host_and_wait.yml](/edit/aitac-automation-handson/solutions/01_add_host_and_wait.yml)

実行方法： `ansible-playbook solutions/01_site.yml`

### 解答例

* [solutions/02_site.yml](/edit/aitac-automation-handson/solutions/02_site.yml)
* [solutions/roles/ec2_instance_create/tasks/main.yml](/edit/aitac-automation-handson/solutions/roles/ec2_instance_create/tasks/main.yml)
* [solutions/roles/ec2_instance_create/defaults/main.yml](/edit/aitac-automation-handson/solutions/roles/ec2_instance_create/defaults/main.yml)
* [solutions/roles/wait_server_boot/tasks/main.yml](/edit/aitac-automation-handson/solutions/roles/wait_server_boot/tasks/main.yml)
* [solutions/roles/httpd_on/tasks/main.yml](/edit/aitac-automation-handson/solutions/roles/httpd_on/tasks/main.yml)
* [solutions/roles/httpd_on/templates/index.html.j2](/edit/aitac-automation-handson/solutions/roles/httpd_on/templates/index.html.j2)
* [solutions/roles/httpd_on_test/tasks/main.yml](/edit/aitac-automation-handson/solutions/roles/httpd_on_test/tasks/main.yml)

実行方法： `ansible-playbook solutions/02_site.yml`

### 解答例

* [solutions/03_site.yml](/edit/aitac-automation-handson/solutions/03_site.yml)
* [solutions/roles/nginx_lb/tasks/main.yml](/edit/aitac-automation-handson/solutions/roles/nginx_lb/tasks/main.yml)
* [solutions/roles/nginx_lb/templates/nginx.conf.j2](/edit/aitac-automation-handson/solutions/roles/nginx_lb/templates/nginx.conf.j2)
* [solutions/roles/nginx_lb_test/tasks/main.yml](/edit/aitac-automation-handson/solutions/roles/nginx_lb_test/tasks/main.yml)

実行方法： `ansible-playbook -i inventories/ec2.py --private-key aitac-automation-key.pem -u centos solutions/03_site.yml`

### 解答例

* [solutions/04_site.yml](/edit/aitac-automation-handson/solutions/04_site.yml)

実行方法： `ansible-playbook solutions/04_site.yml`